In [ ]:
import numpy as np 
import pandas as pd 
import rasterio
import folium
from pyproj import transform, Proj, CRS

from FDSSC import models, utils
from matplotlib import pyplot as plt

In [ ]:
datadir = "./data/LC08_L1TP_098083_20200125_20200128/"
filelist = [files for root, dirs, files in os.walk(datadir)][0]
filelist = [x for x in filelist if "TIF" in x]

In [ ]:
print(filelist)

In [ ]:
dataset = rasterio.open(datadir+filelist[0])

In [ ]:
def find_bbox(dataset):
    tl = dataset.transform * (0, 0)
    tr = dataset.transform * (dataset.width, 0)
    bl = dataset.transform * (0, dataset.height)
    br = dataset.transform * (dataset.width, dataset.height)

    return tl, tr, br, bl

In [ ]:
OCC_df = pd.read_csv('miningSites/processedMS.csv', header=0, index_col=False)
OCC_df.columns = ['MINERAL_DEPOSIT_NO', 'DEPOSIT_COMMODITY_CODES', 'MAJOR_COMMODITY_CODE',
       'MINDEP_CLASS_CODE', 'STATUS_TYPE_VALUE', 'SIZE_TYPE_VALUE',
       'ORE_LITHOLOGY_CODE', 'LATITUDE', 'LONGITUDE']
OCC_df.to_csv('miningSites/processedMS1.csv', index=False)

In [30]:
# read the csv into a pandas data frame
OCC_df = pd.read_csv('miningSites/processedMS.csv', header=0, index_col=False)

OCC_df = OCC_df.loc[OCC_df["MAJOR_COMMODITY_CODE"] == "Au"]


In [ ]:
plt.imshow(dataset.read(1))
plt.show()

In [32]:
m = folium.Map(location=[-31.743944369555788, 138.29644592694788], tiles='OpenStreetMap', zoom_start=7)

inproj, outproj = dataset.crs, Proj('EPSG:4326')

tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri', name = 'Esri Satellite', overlay = False, 
        control = True, opacity=0.7).add_to(m)
image_loc = folium.vector_layers.Polygon(list(zip(*transform(inproj, outproj, *zip(*find_bbox(dataset))))))
image_loc.add_to(m)

for i in OCC_df.index:
    folium.Marker([OCC_df.loc[i,'LONGITUDE'], 
                    OCC_df.loc[i,'LATITUDE']]).add_to(m)

m